In [ ]:
import pylab as plt
import pandas as pd
import glob
import numpy as np
import progressbar
from scipy import interpolate

In [ ]:
files = sorted(glob.glob('/glade/scratch/danieldas/data/Advection/daSilva*'))
files

In [ ]:
particles = {}

for file in files:
    df = pd.read_csv(file, sep=r'\s+', names=['particle_num', 'time', 'radial_dist', 'az_loc', 'energy', 'drift_vel', 'initial_radial_loc', 'initial_az_loc', 'initial_energy', 'first_invariant'])
    bar = progressbar.ProgressBar()

    for _, row in bar(list(df.iterrows())):
        t = int(row.time)

        if t not in particles:
            particles[t] = []

        particles[t].append(row.radial_dist)

In [ ]:
old_particles = {}
fh = open('/glade/scratch/danieldas/data/Advection/traj.LFM-20150317.20210917-174345.200MeVG.lowpass.dat')

for line in fh:
    toks = line.strip().split()
    t = int(float(toks[1]))

    if t not in particles:
        continue

    radial_dist = float(toks[2])

    if t not in old_particles:
        old_particles[t] = []

    old_particles[t].append(radial_dist)

In [ ]:
bins = np.arange(3, 10, .25)
cmap = plt.get_cmap('viridis')

plt.figure(figsize=(18, 4))

for t in sorted(particles.keys()):
    data = np.array(particles[t])
    H, B = np.histogram(data, bins=bins)
    plt.plot(B[:-1] + np.diff(B)/2, H, label=f't={int(t/60):d} min', color=cmap(t/np.max(list(particles.keys()))))

plt.legend()
plt.ylabel('Bin Count', fontsize=18)
plt.xlabel('L* ($R_E$)', fontsize=18)
plt.grid(color='#ccc', linestyle='dashed')
plt.title('L*\'s for traj.LFM-20150317.20210917-174345.200MeVG.lowpass.dat', fontsize=20, fontweight='bold')

In [ ]:
bins = np.arange(3, 10, .25)
cmap = plt.get_cmap('viridis')

plt.figure(figsize=(18, 4))

for t in sorted(particles.keys()):
    data = np.array(old_particles[t])
    H, B = np.histogram(data, bins=bins)
    plt.plot(B[:-1] + np.diff(B)/2, H, label=f't={int(t/60):d} min', color=cmap(t/np.max(list(particles.keys()))))

plt.legend()
plt.ylabel('Bin Count', fontsize=18)
plt.xlabel('Radial Distance ($R_E$)', fontsize=18)
plt.grid(color='#ccc', linestyle='dashed')
plt.title('Radial Distances for traj.LFM-20150317.20210917-174345.200MeVG.lowpass.dat', fontsize=20, fontweight='bold')

In [ ]:
def plot_interp_spect(part_dict, label, title):
    trange = np.arange(1800, dtype=float)
    bins = np.arange(3, 10, .25)
    bin_centers = bins[:-1] + np.diff(bins)/2
    def interp_histogram():
        spect = []

        native_times = set()

        for i, t in enumerate(sorted(part_dict.keys())):
            data = np.array(part_dict[t])
            H, B = np.histogram(data, bins=bins)
            spect.append(H)
            native_times.add(t)

        native_times = sorted(native_times)

        spect = np.array(spect)

        spect_interp = []

        for j in range(spect.shape[1]):
            spline = interpolate.CubicSpline(native_times, spect[:, j])
            spect_interp.append(spline(trange))
        spect_interp = np.array(spect_interp)

        return spect_interp

    plt.figure(figsize=(22, 4))

    spect_interp = interp_histogram()
    plt.pcolor(trange/60, bin_centers, spect_interp)
    plt.colorbar().set_label('Bin Count',  fontsize=18)
    plt.ylabel(label, fontsize=18)
    plt.xlabel('Time (min)',  fontsize=18)
    plt.title(title, fontsize=20, fontweight='bold')

In [ ]:
plot_interp_spect(particles, 'L* ($R_E$)', 'L*\'s for traj.LFM-20150317.20210917-174345.200MeVG.lowpass.dat')

In [ ]:
plot_interp_spect(old_particles, 'Radial Distances ($R_E$)', 'Radial Distances for traj.LFM-20150317.20210917-174345.200MeVG.lowpass.dat')